In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
import json
from sklearn.model_selection import train_test_split

# Load JSON data
with open('data.json', 'r') as f:
    data = json.load(f)

# Extract features and target
time_series = np.array(data['o'])  # Opening prices
n_past = 10  # Length of the input sequence
n_future = 10  # Number of days to predict

# Initialize the scaler and scale the time series data
scaler = MinMaxScaler()
scaled_time_series = scaler.fit_transform(time_series.reshape(-1, 1)).flatten()

# Prepare training data
x_train = []
y_train = []

# Create training data with scaled values
for i in range(len(scaled_time_series) - n_past - n_future):
    x_train.append(scaled_time_series[i:i + n_past])
    y_train.append(scaled_time_series[i + n_past:i + n_past + n_future])

x_train = np.array(x_train).reshape(-1, n_past, 1)  # Reshape to 3D
y_train = np.array(y_train)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

# Build the GAN model
latent_dim = 100

# Generator
generator = keras.Sequential([
    layers.Dense(128, activation='relu', input_dim=latent_dim),
    layers.Dense(n_past, activation='relu'),
    layers.Reshape((n_past, 1))
])

# Discriminator
discriminator = keras.Sequential([
    layers.Flatten(input_shape=(n_past, 1)),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Create GAN
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = keras.Model(gan_input, gan_output)

# Compile GAN
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Training the GAN
batch_size = 2048
epochs = 200

for epoch in range(epochs):
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    generated_samples = generator.predict(noise)

    real_samples = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
    combined_samples = np.concatenate([real_samples, generated_samples])

    labels = np.array([1] * batch_size + [0] * batch_size)
    discriminator_loss = discriminator.train_on_batch(combined_samples, labels)

    gan_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Discriminator Loss: {discriminator_loss[0]}, GAN Loss: {gan_loss}")

# Predict the next 10 days' opening prices with a random noise input
noise = np.random.normal(0, 1, (1, latent_dim))  # Random noise for the generator
predicted_scaled = generator.predict(noise).flatten()  # Flatten the result for easier reading

# Inverse transform to get back original scale
predicted_original_scale = scaler.inverse_transform(predicted_scaled.reshape(-1, 1)).flatten()

# Output the predicted opening prices for the next 10 days
print("Predicted opening prices for the next 10 days (original scale):", predicted_original_scale)


64/64 [==============================] - 0s 1ms/step
Epoch 0, Discriminator Loss: 0.784792959690094, GAN Loss: 0.7300112247467041
64/64 [==============================] - 0s 1ms/step
Epoch 10, Discriminator Loss: 0.6733156442642212, GAN Loss: 0.7125898599624634
64/64 [==============================] - 0s 1ms/step
Epoch 20, Discriminator Loss: 0.6321983337402344, GAN Loss: 0.7453367114067078
64/64 [==============================] - 0s 1ms/step
Epoch 30, Discriminator Loss: 0.6091620922088623, GAN Loss: 0.7623714208602905
64/64 [==============================] - 0s 1ms/step
Epoch 40, Discriminator Loss: 0.6027166843414307, GAN Loss: 0.7629972696304321
64/64 [==============================] - 0s 1ms/step
Epoch 50, Discriminator Loss: 0.6293774843215942, GAN Loss: 0.7417528629302979
64/64 [==============================] - 0s 1ms/step
Epoch 60, Discriminator Loss: 0.6489123702049255, GAN Loss: 0.7218083143234253
64/64 [==============================] - 0s 1ms/step
Epoch 70, Discriminator L